In [ ]:
%matplotlib inline

from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

from numpy import exp, sqrt
from scipy.linalg import expm

Consider the Ornstein-Uhlenbeck process 
$$d X_s = AX_s \mathrm ds + B \mathrm d W_s$$
and its controlled version
$$d X^u_s = AX^u_s \mathrm ds + B u(X_s^u, s) \mathrm ds + B \mathrm d W_s.$$

In [ ]:
np.random.seed(42)

T = 1
d = 2
A = -3 * np.eye(d) + np.random.randn(d, d)
B = np.eye(d)
alpha = np.ones(d)

def g(x):
    return alpha.dot(x)

def u(x, t):
    return -B.T.dot(expm(A.T * (T - t))).dot(alpha)[:, np.newaxis].repeat(x.shape[1], axis=1)

def v(x, t):
    delta_t = 0.001
    N = int(np.floor((T - t) / delta_t)) + 1
    Sigma_n = np.zeros([d, d])
    for t_n in np.linspace(t, T, N):
        Sigma_n += expm(A * t_n).dot(B).dot(B.T).dot(expm(A.T * t_n)) * delta_t
    return (expm(A * (T - t)).dot(x)).dot(alpha) - 0.5 * alpha.T.dot(Sigma_n.dot(alpha))

In [ ]:
def importance_sampling(K, delta_t, epsilon=0, verbose=True, T_perturb=T):

    sq_delta_t = np.sqrt(delta_t)
    N = int(T / delta_t)

    X = np.zeros([d, K])
    X_u = np.zeros([d, K])
    ito_int = np.zeros(K)
    riemann_int = np.zeros(K)
    f_int = np.zeros(K)
    f_int_u = np.zeros(K)

    for n in range(N + 1):
        xi = np.random.randn(d, K)
        X = X + A.dot(X) * delta_t + B.dot(xi) * sq_delta_t
        if n * delta_t <= T_perturb:
            ut = u(X_u, n * delta_t) + epsilon
        else:
            ut = u(X_u, n * delta_t)
        X_u = X_u + (A.dot(X_u) + B.dot(ut)) * delta_t + B.dot(xi) * sq_delta_t
        ito_int += np.sum(ut * xi, 0) * sq_delta_t
        riemann_int += np.sum(ut**2, 0) * delta_t

    girsanov = np.exp(- ito_int - 0.5 * riemann_int)

    stats = {}
    stats['naive'] = {}
    stats['IS'] = {}
    stats['naive']['mean'] = np.mean(np.exp(- f_int - g(X)))
    stats['naive']['variance'] = np.var(np.exp(- f_int - g(X)))
    stats['naive']['relative_error'] = np.sqrt(stats['naive']['variance']) / stats['naive']['mean']
    stats['IS']['mean'] = np.mean(np.exp(- f_int_u - g(X_u)) * girsanov)
    stats['IS']['variance'] = np.var(np.exp(- f_int_u - g(X_u)) * girsanov)
    stats['IS']['relative_error'] = np.sqrt(stats['IS']['variance']) / stats['IS']['mean']
    
    if verbose:
        print('naive mean: %.4e, naive variance: %.4e, naive RE: %.4e' % (stats['naive']['mean'], stats['naive']['variance'], stats['naive']['relative_error']))
        print('IS mean:    %.4e, IS variance:    %.4e, IS RE:    %.4e' % (stats['IS']['mean'], stats['IS']['variance'], stats['IS']['relative_error']))
        print('true mean:  %.4e' % np.exp(-v(np.zeros(d), 0)))
    return stats

## constant perturbation

In [ ]:
# vary epsilon

epsilon_range = np.linspace(0, 1.2, 10)

RE = []

for epsilon in epsilon_range:
    stats = importance_sampling(K=1000000, delta_t=0.01, epsilon=epsilon, verbose=False)
    RE.append(stats['IS']['relative_error'])

In [ ]:
# vary d

RE_d = []

d_range = np.arange(2, 15)

for d in d_range:
    # problem: OU
    T = 1
    A = -3 * np.eye(d) + np.random.randn(d, d)
    B = np.eye(d)
    alpha = np.ones(d)

    def g(x):
        return alpha.dot(x)

    def u(x, t):
        return -B.T.dot(expm(A.T * (T - t))).dot(alpha)[:, np.newaxis].repeat(x.shape[1], axis=1)

    def v(x, t):
        delta_t = 0.001
        N = int(np.floor((T - t) / delta_t)) + 1
        Sigma_n = np.zeros([d, d])
        for t_n in np.linspace(t, T, N):
            Sigma_n += expm(A * t_n).dot(B).dot(B.T).dot(expm(A.T * t_n)) * delta_t
        return (expm(A * (T - t)).dot(x)).dot(alpha) - 0.5 * alpha.T.dot(Sigma_n.dot(alpha))
    
    stats = importance_sampling(K=1000000, delta_t=0.01, epsilon=0.5, verbose=False)
    RE_d.append(stats['IS']['relative_error'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))
ax[0].plot(epsilon_range, RE, label='sampled');
ax[0].plot(epsilon_range, np.sqrt(np.exp(2 * epsilon_range**2 * T) - 1), '--', label='computed');
ax[0].set_xlabel(r'$\varepsilon$')
ax[0].set_ylabel(r'$r(u)$')
ax[0].set_title(r'Ornstein-Uhlenbeck, $d = 2$')
ax[0].legend()
ax[1].plot(d_range, RE_d);
ax[1].plot(d_range, np.sqrt(np.exp(d_range * 0.5**2 * T) - 1), '--');
ax[1].set_xlabel(r'$d$')
ax[1].set_ylabel(r'$r(u)$');
ax[1].set_title(r'Ornstein-Uhlenbeck, $\varepsilon = 0.5$');

#fig.savefig('img/suboptimal_control_OU_eps_d_2.pdf')

## time-dependent perturbation (need to rerun cell with matrices A, B etc.)

In [ ]:
# vary epsilon

epsilon_range_time = np.linspace(0, 2.6, 10)

RE_time = []

for epsilon in epsilon_range_time:
    stats = importance_sampling(K=10000, delta_t=0.01, epsilon=epsilon, verbose=False, T_perturb=0.2)
    RE_time.append(stats['IS']['relative_error'])

In [ ]:
# vary d

RE_time_d = []

d_range_time = np.arange(2, 20)

for d in d_range:
    # problem: OU
    T = 1
    A = -3 * np.eye(d) + np.random.randn(d, d)
    B = np.eye(d)
    alpha = np.ones(d)

    def g(x):
        return alpha.dot(x)

    def u(x, t):
        return -B.T.dot(expm(A.T * (T - t))).dot(alpha)[:, np.newaxis].repeat(x.shape[1], axis=1)

    def v(x, t):
        delta_t = 0.001
        N = int(np.floor((T - t) / delta_t)) + 1
        Sigma_n = np.zeros([d, d])
        for t_n in np.linspace(t, T, N):
            Sigma_n += expm(A * t_n).dot(B).dot(B.T).dot(expm(A.T * t_n)) * delta_t
        return (expm(A * (T - t)).dot(x)).dot(alpha) - 0.5 * alpha.T.dot(Sigma_n.dot(alpha))
    
    stats = importance_sampling(K=1000000, delta_t=0.01, epsilon=1.0, verbose=False, T_perturb=0.2)
    RE_time_d.append(stats['IS']['relative_error'])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))
ax[0].plot(epsilon_range, RE_time, label='sampled');
ax[0].plot(epsilon_range, np.sqrt(np.exp(2 * epsilon_range**2 * 0.2) - 1), '--', label='computed');
ax[0].set_xlabel(r'$\varepsilon$')
ax[0].set_ylabel(r'$r(u)$')
ax[0].set_title(r'Ornstein-Uhlenbeck, $d = 2$')
ax[0].legend()
ax[1].plot(d_range, RE_time_d);
ax[1].plot(d_range, np.sqrt(np.exp(d_range * 1.0**2 * 0.2) - 1), '--');
ax[1].set_xlabel(r'$d$')
ax[1].set_ylabel(r'$r(u)$');
ax[1].set_title(r'Ornstein-Uhlenbeck, $\varepsilon = 1.0$');

#fig.savefig('img/suboptimal_control_OU_time_dependent_eps_d_2.pdf')

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 10}

plt.rc('font', **font)

fig, ax = plt.subplots(1, 4, figsize=(15, 3.5))

fig.suptitle('Ornstein-Uhlenbeck process', fontsize=16)

ax[0].plot(epsilon_range, RE, label='sampled');
ax[0].plot(epsilon_range, np.sqrt(np.exp(2 * epsilon_range**2) - 1), '--', label='computed');
ax[0].plot(epsilon_range, np.sqrt(np.exp(0.5 * 2 * epsilon_range**2) - 1), '--', label='KL bound');
ax[0].set_xlabel(r'$\varepsilon$')
ax[0].set_ylabel(r'$r(u)$')
ax[0].set_title(r'constant perturbation, $d = 2$')
ax[0].legend()
ax[1].plot(d_range, RE_d);
ax[1].plot(d_range, np.sqrt(np.exp(d_range * 0.5**2) - 1), '--');
ax[1].plot(d_range, np.sqrt(np.exp(0.5 * d_range * 0.5**2) - 1), '--');
ax[1].set_xlabel(r'$d$')
ax[1].set_ylabel(r'$r(u)$');
ax[1].set_xticks(np.linspace(3, d_range[-1] - 2, 4))

ax[1].set_title(r'constant perturbation, $\varepsilon = 0.5$');
ax[2].plot(epsilon_range_time, RE_time, label='sampled');
ax[2].plot(epsilon_range_time, np.sqrt(np.exp(2 * epsilon_range_time**2 * 0.2) - 1), '--', label='computed');
ax[2].plot(epsilon_range_time, np.sqrt(np.exp(0.5 * 2 * epsilon_range_time**2 * 0.2) - 1), '--', label='KL bound');
ax[2].set_xlabel(r'$\varepsilon$')
ax[2].set_ylabel(r'$r(u)$')
ax[2].set_title(r'time-dependent perturbation, $d = 2$')
ax[3].plot(d_range_time, RE_time_d);
ax[3].plot(d_range_time, np.sqrt(np.exp(d_range_time * 1.0**2 * 0.2) - 1), '--');
ax[3].plot(d_range_time, np.sqrt(np.exp(0.5 * d_range_time * 1.0**2 * 0.2) - 1), '--');
ax[3].set_xlabel(r'$d$')
ax[3].set_ylabel(r'$r(u)$');
ax[3].set_title(r'time-dependent perturbation, $\varepsilon = 1.0$');

fig.tight_layout(rect=[0, 0.03, 1, 0.88])

#fig.savefig('img/suboptimal_control_OU_eps_const_time_d_2_KL.pdf')